# Using loaders

:::{note}
You are viewing a working draft with expected completion in early 2025.
:::

In the previous sections, you learned how to handle both individual images and entire collections of images and related meta-data via datasets. To train medical imaging AI models, however, it is necessary to combine your data records into a single array / tensor called a "batch". The `nitrain.Loader` class is the way to accomplish this task in a flexible and powerful way.

In this chapter, we will go through the basic ways to generate trainable batches of data from your dataset. In future chapters in this section, we will expand on these basic examples by adding samplers and random augmentation transforms.


### Prerequisites

In this chapter, we will use the standard libraries of nitrain, ants, and numpy. As before, we will be primarily using images already loaded into memory but the concepts still apply to reading images from local folders or the cloud.

In [1]:
import nitrain as nt
import ants
import numpy as np

## Initialization

To create a loader, you pass a nitrain dataset into the `nitrain.Loader` class. There are a few other parameters which can be set to control how the loader serves batches, but in general the only required parameter is the `images_per_batch` argument. In general, this determines how many records are loaded per batch. 

Let's see how this looks like with an in-memory example. First, we create a dataset with 2D images as input and a scalar value label as output.

In [2]:
images = [ants.from_numpy(np.ones((100,100)))*i for i in range(5)]
labels = list(range(5))

dataset = nt.Dataset(images, labels)

Then, we pass the dataset into the loader class while also setting the image per batch argument.

In [6]:
loader = nt.Loader(dataset, images_per_batch=4)

xb, yb = next(iter(loader))
print(type(xb), ' | ', type(yb))
print(xb.shape, ' | ', yb.shape)

<class 'numpy.ndarray'>  |  <class 'numpy.ndarray'>
(4, 100, 100, 1)  |  (4,)


You should notice two important things about loaders that differ from datasets. Firstly, you cannot index loaders like datasets because loaders are python iterators. Therefore, you must call `next(iter(loader))` or loop through the loader with a for-loop to access batches. The iterator will end once all images have been served once. 

Next, notice that the result of a loader is a numpy array and not an ants image. That is because, as mentioned, AI model training requires arrays or tensors for training.

Since we set `images_per_batch` equal to 4, we get 4 images and labels together in arrays when we grab the next iteration. This is a completely trainable batch of data, but we can do a lot more with it as you'll see later.

## Iteration

As mentioned, you can loop through the entire dataset to generate batches. That looks like this:

In [9]:
for xb, yb in loader:
    print(xb.shape)

(4, 100, 100)
(1, 100, 100)


Because our dataset only had 5 records and our images per batch was 4, we only had two batches. The final batch will always be equal to or less than the size of the batch size.

## Controlling channels

We saw that the `images_per_batch` parameter was needed to create a loader, as it controlled the size of the batch in the basic case. There are other parameters which control the loader as well.

Firstly, did you notice that the shape of the input batch of images was `(4,100,100,1)`? That extra "1" at the end is called the "channel" and it was added automatically by the nitrain loader because nearly all AI frameworks require at least one channel. In other words, a 3D image would have a shape of (4,100,100,100,1) if put into a batch too.

That parameter can be controlled via the `channels_first` argument in the loader. It defaults to False and has the following options: True, False, and None. Let's see what happens if you change that argument. Setting it to True looks like this:

In [7]:
loader = nt.Loader(dataset, images_per_batch=4, channels_first=True)

xb, yb = next(iter(loader))
print(xb.shape, ' | ', yb.shape)

(4, 1, 100, 100)  |  (4,)


As you see, in this case the added channel comes first in the image. This is the format that some framework such as Pytorch expect. And setting it to None looks like this:

In [8]:
loader = nt.Loader(dataset, images_per_batch=4, channels_first=None)

xb, yb = next(iter(loader))
print(xb.shape, ' | ', yb.shape)

(4, 100, 100)  |  (4,)


In this case, there was no channel added. Therefore, you have full control over added channels when generating batches. Importantly, you should note that channels are only added to image records and they will not be added if an image already has components.

## Combined readers

The example until now has been with only one image as input and one value as output. As demonstrated in the chapters about datasets, you can have arbitarily complex combinations of inputs and outputs. Passing more complicated datasets into loaders has the exact functionality you would expect.

In [11]:
images = [ants.from_numpy(np.ones((100,100)))*i for i in range(5)]
labels = list(range(5))

dataset = nt.Dataset([images, images], labels)
loader = nt.Loader(dataset, images_per_batch=4)

xb, yb = next(iter(loader))

print(xb[0].shape)
print(xb[1].shape)

(4, 100, 100, 1)
(4, 100, 100, 1)


Because our dataset has two inputs, the batches will also have two inputs. The result of iterating through the loader on the input size is therefore a list of two arrays rather than an array.

## Summary


In this chapter, you saw how to generate batches from datasets in the simplest case. There are two important features of loaders that we did not address here: using samplers to generated batches of sub-images (e.g., slices, patches, or blocks) and using random transforms to create augmented datasets. 

The next two chapters will go further in-depth on these two features so that you can get the most out of batch generation and training.